In [ ]:
#import openai
import weave
weave.use_frontend_devmode()
from weave.monitoring import openai, init_monitor
from weave.panels_py import panel_openai_monitor

In [ ]:
m = init_monitor('shawn/oai-mon/test6')

In [ ]:
result = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[
        {"role": "system", "content": "You love to call functions"},
        {"role": "user", "content": f"what's the weather today"},
    ])
result

In [ ]:
# Streaming flavor
# result_iter = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[
#         {"role": "system", "content": "You love to call functions"},
#         {"role": "user", "content": f"what's the weather today"},
#         #{"role": "user", "content": f"Please write a long story"},
#     ], stream=True)
# result_iter
# for row in result_iter:
#     pass

In [ ]:
# TODO: restore async variant
# result = await monitored_a_create(model="gpt-3.5-turbo", messages=[
#         {"role": "system", "content": "You are a world-class machine learning researcher."},
#         {"role": "user", "content": f"Please provide a simple, fact-based question to send to an AI system. Do not say anything other than the question itself. Use this random number as inspiration: {random.random()}."},
#     ])
# result

In [ ]:
# x = weave.use(m._streamtable.rows()['inputs']['messages'][0])
# x

In [ ]:
#m._streamtable.rows()

In [ ]:
board = panel_openai_monitor.board.raw_resolve_fn(m.rows())

In [ ]:
board

In [ ]:
# TODO: ensure works with function calls
# functions = [
#     {
#         "name": "get_current_weather",
#         "description": "Get the current weather",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "The city and state, e.g. San Francisco, CA",
#                 },
#                 "format": {
#                     "type": "string",
#                     "enum": ["celsius", "fahrenheit"],
#                     "description": "The temperature unit to use. Infer this from the users location.",
#                 },
#             },
#             "required": ["location", "format"],
#         },
#     },
#     {
#         "name": "get_n_day_weather_forecast",
#         "description": "Get an N-day weather forecast",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "The city and state, e.g. San Francisco, CA",
#                 },
#                 "format": {
#                     "type": "string",
#                     "enum": ["celsius", "fahrenheit"],
#                     "description": "The temperature unit to use. Infer this from the users location.",
#                 },
#                 "num_days": {
#                     "type": "integer",
#                     "description": "The number of days to forecast",
#                 }
#             },
#             "required": ["location", "format", "num_days"]
#         },
#     },
# ]

# result = openai.ChatCompletion.create(model="gpt-3.5-turbo", functions=functions, messages=[
#         {"role": "system", "content": "You love to call functions."},
#         {"role": "user", "content": f"what's the weather today"},
#     ])